In [4]:
import tensorflow as tf
import numpy as np
from PIL import Image

from keras.models import load_model
from keras.utils import load_img
from keras.utils import img_to_array

# idee: diesem python programm wird eine projekt id vom Frontend übergeben, das programm nimmt daraufhin die datenbank zur hand (nützliches tool
# zum öffnen der Datenbank: https://sqlitebrowser.org/dl/) öffnet die Bilddatentabelle und entnimmt 
# dieser welche Bilder im Ordner "material images" die projekt id haben, öffnet diese
# und ermittelt den Wert. Diser wird wieder in die Bilddatentabelle eingetragen und kann so vom Frontend weiterverwendet werden

# load model, insert in the folder first
interpreter = tf.keras.models.load_model('material_model.h5')

# get the right image
img = tf.keras.utils.load_img('assets/material_images/1.jpg', target_size=(400, 300))

x = tf.keras.utils.img_to_array(img, dtype="float32")

# predict from image
prediction = interpreter.predict(x)


print(prediction)

ValueError: in user code:

    File "C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\nicol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1791, in pr